# 循环神经网络  
本节介绍循环神经网络，下图展示了如何基于循环神经网络实现语言模型。我们的目的是基于当前的输入与过去的输入序列，预测序列的下一个字符。循环神经网络引入一个隐藏变量$H$，用$H_{t}$表示$H$在时间步$t$的值。$H_{t}$的计算基于$X_{t}$和$H_{t-1}$，可以认为$H_{t}$记录了到当前字符为止的序列信息，利用$H_{t}$对序列的下一个字符进行预测。

## 循环神经网络的构造

我们先看循环神经网络的具体构造。假设$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$是时间步$t$的小批量输入，$\boldsymbol{H}_t  \in \mathbb{R}^{n \times h}$是该时间步的隐藏变量，则：


$$
\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).
$$


其中，$\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$，$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$，$\boldsymbol{b}_{h} \in \mathbb{R}^{1 \times h}$，$\phi$函数是非线性激活函数。由于引入了$\boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}$，$H_{t}$能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。由于$H_{t}$的计算基于$H_{t-1}$，上式的计算是循环的，使用循环计算的网络即循环神经网络（recurrent neural network）。

在时间步$t$，输出层的输出为：


$$
\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.
$$


其中$\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$，$\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。


## 从零开始实现循环神经网络

我们先尝试从零开始实现一个基于字符级循环神经网络的语言模型，这里我们使用周杰伦的歌词作为语料，首先我们读入数据：

In [1]:
import torch
import torch.nn as nn
import time
import math
import sys
sys.path.append('../')
import d2l_jay9460 as d2l
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### one-hot向量

我们需要将字符表示成向量，这里采用one-hot向量。假设词典大小是$N$，每次字符对应一个从$0$到$N-1$的唯一的索引，则该字符的向量是一个长度为$N$的向量，若字符的索引是$i$，则该向量的第$i$个位置为$1$，其他位置为$0$。下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [2]:
def one_hot(x, n_class, dtype=torch.float32):
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    result.scatter_(1, x.long().view(-1,1), 1) #result[i, x[i,0]] = 1 第一个参数dim填写的维度，1按行填写；第二个参数为位置，即下标；第三个参数为填写的数值
    return result
x = torch.tensor([0., 2])
x_one_hot = one_hot(x, vocab_size)
print(x_one_hot)
print(x_one_hot.shape)
print(x_one_hot.sum(dim=1)) #axis写错了，dim才对

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])
torch.Size([2, 1027])
tensor([1., 1.])


我们每次采样的小批量的形状是（批量大小, 时间步数）。下面的函数将这样的小批量变换成数个形状为（批量大小, 词典大小）的矩阵，矩阵个数等于时间步数。也就是说，时间步$t$的输入为$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中$n$为批量大小，$d$为词向量大小，即one-hot向量长度（词典大小）。

In [3]:
def to_onehot(X, n_class):
    return [one_hot(X[:,i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(X)
print(inputs)
print(len(inputs), inputs[0].shape)


tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])
[tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
5 torch.Size([2, 1027])


### 初始化模型参数

In [4]:
num_inputs = vocab_size
num_hiddens = 256
num_outputs = vocab_size

def get_params():
    def _one(shape):
        param = torch.zeros(shape, device=device, dtype=torch.float32)
        nn.init.normal_(param, 0, 0.01)
        return torch.nn.Parameter(param)
    
    #隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device))
    
    #输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device))
    return (W_xh, W_hh, b_h, W_hq, b_q)

### 定义模型  
函数rnn用循环的方式依次完成循环神经网络每个时间步的计算

In [5]:
def rnn(inputs, state, params):
    #inputs 和 outputs皆为num_steps（时间步数）个形状为（batch_size, vocab_size）的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y) #???????????????????????
    return outputs, (H,)

In [6]:
#函数init_rnn_state初始化隐藏变量，这里的返回值是一个元组。
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [7]:
print(X.shape)# batch_size为2，时间步为5
print(num_hiddens)
print(vocab_size)
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(inputs), inputs[0].shape) #输入长度为时间步，大小为（batch_size, 词典长度）
print(len(outputs), outputs[0].shape)
print(len(state), state[0].shape)
print(len(state_new), state_new[0].shape)

torch.Size([2, 5])
256
1027
5 torch.Size([2, 1027])
5 torch.Size([2, 1027])
1 torch.Size([2, 256])
1 torch.Size([2, 256])


### 裁剪梯度

循环神经网络中较容易出现梯度衰减或梯度爆炸，这会导致网络几乎无法训练。裁剪梯度（clip gradient）是一种应对梯度爆炸的方法。假设我们把所有模型参数的梯度拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是$\theta$。裁剪后的梯度


$$
 \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}
$$


的$L_2$范数不超过$\theta$。

In [8]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta/norm)

### 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [9]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device) #batch_size为1
    output = [char_to_idx[prefix[0]]] #记录 prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix)-1):
        #将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]],device=device), vocab_size)
        #计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(Y[0].argmax(dim=1).item())
    return '-'.join([idx_to_char[i] for i in output])

In [10]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分-开-师-沉-年-习-阵-思-物-到-告-涌'

### 困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下[“softmax回归”](../chapter_deep-learning-basics/softmax-regression.ipynb)一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

### 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。

In [55]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            # inputs是num_steps个形状为(batch_size, vocab_size)的矩阵
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成形状为
            # (num_steps * batch_size,)的向量，这样跟输出的行一一对应
            #print(Y.shape)
            y = torch.flatten(Y.t()) #这里修改一下
            #y = torch.flatten(Y.T)
            #print(y.shape)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))
    

In [44]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [57]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 68.268988, time 2.49 sec
 - 分-开- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我-不-要-再-想- -我
 - 不-分-开- - -我-的-要-我-想-狂-的-可-爱-女-人- -坏-坏-的-让-我-疯-狂-的-可-爱-女-人- -坏-坏-的-让-我-疯-狂-的-可-爱-女-人- -坏-坏-的-让-我-疯-狂-的-可-爱
epoch 100, perplexity 9.922735, time 2.11 sec
 - 分-开- -我-想-想-你-想-你- -不-知-不-觉- -你-已-经-一-个-秋- -后-哼-哈- -一-只-两- -我-想-就-这-样-牵- -你-的-手-不-对-口- -如-果-我- -别-你-是-那
 - 不-分-开-永- -我-不-能-再-想- -我-不- -我-不- -我-不-能- -爱-情-我-的-不-爸- -有-话-是-对-医- -你-在-人-人-抽- - -分-什-么- -其-直-我- -一-子-球- 
epoch 150, perplexity 2.991332, time 2.21 sec
 - 分-开- -有-杰-段- -旧-皮-箱- -我-来-就-这-样-牵-着- -一-只-好-酒- -一-颗-心-外-的-溪-边- -默-默-等-待- -娘-子- -一-壶-好-酒- -再-是-我-跟-已-得-猜
 - 不-分-开-吗- -我-爱-能-爸- -你-檐-走-外- -说-人-海-中-夫- -这-的-躺-美- -我-想-要-你-我-的-我-面-封- -难-道-你-手- -我-想-好-爸- -是-颗-不-外- -没-人
epoch 200, perplexity 1.607664, time 2.19 sec
 - 分-开- -装-蟑-在-双-截- -静-让-盘-停-留- -有-窝-在-角-落- -这-样-就-直-说- -这-样-在-角-落- -不-爽-就-反-驳- -到-底-什-么-奇- -三-蝪-横-反-走- -这
 - 不-分-开-吗- -我-叫-你-爸- -你-打-我-妈- -这-样-对-吗- -硬-人-海-中- -都-目-没-种-

In [56]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 63.188633, time 1.59 sec
 - 分-开- -我-想-要-这-你-的-让-我- -坏-成-我-有- -你-谁-一-直- -我-想-一-场- -我-想-我-的-可-头-女-人- -坏-坏-的-让-我-疯- -的-可-爱-兮- -快-使-用-双
 - 不-分-开- -我-不-要-的-可-写-女-人- -坏-坏-的-让-我-疯- -的-可-爱-兮- -快-使-用-双- -我-想-一-直- -我-想-一-场- -我-想-我-的-可-头-女-人- -坏-坏-的-让
epoch 100, perplexity 7.344551, time 2.00 sec
 - 分-开- -我-不-要-再-想- -我-不-能-再-想- -我-不- -我-不- -我-不-要-再-想-你- -爱-情-我-的-见-快-就-像-龙-卷-风- -离-不-开-暴-风-圈-来-不-的-见- - 
 - 不-分-开-柳- -你-已-经-这-节- - -后-有-不-觉-我- -你-拿-着-球-不-球- -又-不-底- -一-直-走- -我-想-就-这-样-牵-着-你-的-手-不-放-开- -爱-可-不-可-以-简
epoch 150, perplexity 2.096177, time 2.19 sec
 - 分-开- -我-给-的- -你-怎-么- -装-满-了-明-的-片-牌- -干-什-么- -干-什-么- -我-像-我-这-话-踢-面- -你-什-么- -干-什-么- -停-行-四-明-的-招-牌- -干
 - 不-分-开-觉- -你-已-经-离-开-我- -不-知-不-觉- -我-跟-了-这-节-奏- -后-知-后-觉- -又-过-了-一-个-秋- -后-知-后-觉- -我-该-好-好-生-活- -我-该-好-好-生
epoch 200, perplexity 1.289615, time 2.02 sec
 - 分-开- -我-给-上- -你-怎-么- -我-满-了-任-督-布-脉- -干-什-么- -干-什-么- -东-被-我-一-脚-踢-开- -快-使-用-双-截-棍- -哼-哼-哈-兮- -快-使-用-双-截
 - 不-分-开-觉- -你-已-经-离-开-我- -不-知-不-觉- -我-跟-了-这-节-奏- -后-知-后-觉- -

## 循环神经网络的简洁实现  

### 定义模型

我们使用Pytorch中的`nn.RNN`来构造循环神经网络。在本节中，我们主要关注`nn.RNN`的以下几个构造函数参数：

* `input_size` - The number of expected features in the input x
* `hidden_size` – The number of features in the hidden state h
* `nonlinearity` – The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
* `batch_first` – If True, then the input and output tensors are provided as (batch_size, num_steps, input_size). Default: False

这里的`batch_first`决定了输入的形状，我们使用默认的参数`False`，对应的输入形状是 (num_steps, batch_size, input_size)。

`forward`函数的参数为：

* `input` of shape (num_steps, batch_size, input_size): tensor containing the features of the input sequence. 
* `h_0` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided. If the RNN is bidirectional, num_directions should be 2, else it should be 1.

`forward`函数的返回值是：

* `output` of shape (num_steps, batch_size, num_directions * hidden_size): tensor containing the output features (h_t) from the last layer of the RNN, for each t.
* `h_n` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the hidden state for t = num_steps.

现在我们构造一个`nn.RNN`实例，并用一个简单的例子来看一下输出的形状。

In [61]:
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35,2

X = torch.rand(num_steps, batch_size, vocab_size)
state = None
Y, state_new = rnn_layer(X, state)
print(Y.shape, state_new.shape)

torch.Size([35, 2, 256]) torch.Size([1, 2, 256])


### 定义完整的基于循环神经网络的语言模型

In [65]:
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) #如果是双向的，那么就是2否则为1，这里为1
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size) #定义线性层构造输出函数
        
    def forward(self, inputs, state):
        # inputs.shape: (batch_size, num_steps)
        X = to_onehot(inputs, vocab_size)
        X = torch.stack(X) #把X的各个元素堆叠起来，X.shape:(num_steps, batch_size, vocab_size)
        
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])
        
        output = self.dense(hiddens)
        return output, state

In [66]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]]  # output记录prefix加上预测的num_chars个字符
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])

In [67]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开鹿戒戒梭在办墙景错呵'

In [70]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state[0].detach_()
                    state[1].detach_()
                else: 
                    state.detach_()
            (output, state) = model(X, state) # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.t())
            l = loss(output, y.long())
            
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [71]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 11.619536, time 0.45 sec
 - 分开 我不了 想要我不想你  我不了我想你我不要我不 我不要再想 我不 我不 我不要再想 我不要再想 我
 - 不分开 我想你你想你我不 我不你我想 我不能再想 我不 我不 我不要再想 我不要再想 我不 我不 我不要再
epoch 100, perplexity 1.335515, time 0.56 sec
 - 分开始一句 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈
 - 不分开 说着了不那  不想活 说你就是你不了 别不会 原过就只别人眼睛看着我 别发抖 快给我抬起头 有话去
epoch 150, perplexity 1.072169, time 0.47 sec
 - 分开 一句 停留的带刚 不要再河前再到 沙漠之中怎么会有泥鳅 话说完飞过一只海鸥 大峡谷的风呼啸而过 是
 - 不分开 你在 不不的 爱你怎么面痛我都能为力 我不懂 别的太多 我有一直到当 靠有你人想要 这种当  说你
epoch 200, perplexity 1.034750, time 0.47 sec
 - 分开 一句 停格的客天 我 店看着我 家乡的人我叫我感动的可以女人 坏坏的让我疯狂的可爱女人 坏坏的让我
 - 不分开 你在 不不的 爱你怎么面痛我 甩开球我满腔的怒火 我想揍你 一个人 后知不觉 你已经离开我 不知不
epoch 250, perplexity 1.021353, time 0.37 sec
 - 分开 一句连停留 带一直到老 我 别发抖 快给我抬起头 有话去对医药箱说 别怪我 别怪我 说你怎么面对我
 - 不分开 你是我不想 你说 我有一这里我的动的一枚铜币 悲伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 让
